# Лабораторна робота №1
## Поліни Олефір
Посилання: сторінка [даних для України](https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/vh_browseByCountry_province.php?country_code=UKR&province_id=1) та зразок [завантаження даних](https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_provinceData.php?country=UKR&provinceID=10&year1=2017&year2=2018&type=Mean) для вибраної області.

Для початку завантажимо csv файл з оригінальними індексами областей, та допишемо в нього нові індекси, колонка `newID`.

In [10]:
import pandas as pd
from sys import stdout as out

provinces = pd.read_csv('ukr_provinces.csv', comment='#', sep='[, ]+', engine='python')
provinces[['province_name','newID','provinceID']][5:-5]

,province_name,newID,provinceID
5,Donetsk,4,6
6,Ivano-Frankivsk,8,7
7,Kharkiv,19,8
8,Kherson,20,9
9,Khmelnytskyy,21,10
10,Kiev,9,11
11,Kiev(City),26,12
12,Kirovohrad,10,13
13,Luhansk,11,14
14,Lviv,12,15


> Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. Ця процедура має бути автоматизована, параметром процедури має бутиіндекс (номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження;

Процедуру реалізовано як метод `get_province_data()` класу `DataLoader` в [окремому файлі](./utils.py). Продемонструємо використання - файл буде завантажено в папку [./data](./data "Шлях до папки є атрибутом класу DataLoader"):

In [11]:
import utils
from importlib import reload
reload(utils) # https://stackoverflow.com/q/1254370

# Years for the data range
year_from = 2014
year_to   = 2017

with utils.DataLoader() as loader:
    loader.get_province_data(1, year_from, year_to)

Тепер завантажимо файли для усіх адміністративних одиниць України:

In [12]:
echo = "Loading province {:2} (newID {:2}), {:17}...  "  

with utils.DataLoader() as loader:
    for index, province in provinces.iterrows():
        out.write(echo.format(province['provinceID'], province['newID'], province['province_name']))
        loader.get_province_data(province['provinceID'], year_from, year_to)
        out.write("Done.\n")

Loading province  1 (newID 22), Cherkasy         ...  Done.
Loading province  2 (newID 24), Chernihiv        ...  Done.
Loading province  3 (newID 23), Chernivtsi       ...  Done.
Loading province  4 (newID 25), Crimea           ...  Done.
Loading province  5 (newID  3), Dnipropetrovsk   ...  Done.
Loading province  6 (newID  4), Donetsk          ...  Done.
Loading province  7 (newID  8), Ivano-Frankivsk  ...  Done.
Loading province  8 (newID 19), Kharkiv          ...  Done.
Loading province  9 (newID 20), Kherson          ...  Done.
Loading province 10 (newID 21), Khmelnytskyy     ...  Done.
Loading province 11 (newID  9), Kiev             ...  Done.
Loading province 12 (newID 26), Kiev(City)       ...  Done.
Loading province 13 (newID 10), Kirovohrad       ...  Done.
Loading province 14 (newID 11), Luhansk          ...  Done.
Loading province 15 (newID 12), Lviv             ...  Done.
Loading province 16 (newID 13), Mykolayiv        ...  Done.
Loading province 17 (newID 14), Odessa  

> Зчитати завантажені текстові файли у фрейм (детальніше про роботу із фреймами буде розказано у подальших лабораторних роботах). Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

Відкриємо папку [./data](./data) та поглянемо на завантажені файли. Помітимо, що попри заголовок файлу, дані не містять номеру області, тож єдиним джерелом номеру є ім'я файлу та номер треба буде додавати до фрейму при зчитуванні.

In [13]:
import os
import re

def read_province_data(data_dir='./data', pattern='P[0-9]+-[0-9]+-[0-9]+.csv'):
    
    if not os.path.exists(data_dir): # Exit if the folder does not exist.
        out.write("Folder {} not found!\n".format(data_dir))
        return None
    
    regexp = re.compile(pattern)
    data_frame = pd.DataFrame()
    
    for name in (n for n in os.listdir(data_dir) if regexp.match(n)):
        file_path = os.path.join(data_dir, name)
        if os.path.isfile(file_path):
            out.write("Reading {}... ".format(name))

            # Detect province ID from the file name
            province_id = int(re.search('P([0-9]+)', name).group(1))
            
            # Read province data frame from the file
            province_frame = pd.read_csv(file_path, sep='[, ]+', engine='python', #index_col=[0,1],
                                         names=['Year','Week','SMN','SMT','VCI','TCI','VHI'])
            
            # Add column with province_id
            province_frame['Province'] = province_id
            
            # Append the resulting data frame
            data_frame = data_frame.append(province_frame)
            
            out.write("Province ID: {}\n".format(province_id))
    
    return data_frame
    
data_frame = read_province_data()
data_frame

Reading P01-180308-213122.csv... Province ID: 1
Reading P02-180308-213122.csv... Province ID: 2
Reading P03-180308-213123.csv... Province ID: 3
Reading P04-180308-213124.csv... Province ID: 4
Reading P05-180308-213124.csv... Province ID: 5
Reading P06-180308-213125.csv... Province ID: 6
Reading P07-180308-213125.csv... Province ID: 7
Reading P08-180308-213126.csv... Province ID: 8
Reading P09-180308-213126.csv... Province ID: 9
Reading P10-180308-213127.csv... Province ID: 10
Reading P11-180308-213127.csv... Province ID: 11
Reading P12-180308-213128.csv... Province ID: 12
Reading P13-180308-213128.csv... Province ID: 13
Reading P14-180308-213129.csv... Province ID: 14
Reading P15-180308-213129.csv... Province ID: 15
Reading P16-180308-213130.csv... Province ID: 16
Reading P17-180308-213131.csv... Province ID: 17
Reading P18-180308-213131.csv... Province ID: 18
Reading P19-180308-213132.csv... Province ID: 19
Reading P20-180308-213132.csv... Province ID: 20
Reading P21-180308-213133.csv

,Year,Week,SMN,SMT,VCI,TCI,VHI,Province
0,2014,1,0.070,264.20,59.34,26.13,42.74,1
1,2014,2,0.067,262.61,58.73,30.85,44.79,1
2,2014,3,0.066,261.66,57.73,35.26,46.49,1
3,2014,4,0.068,261.93,58.86,35.85,47.36,1
4,2014,5,0.073,263.23,60.34,34.37,47.35,1
5,2014,6,0.081,265.70,61.13,31.44,46.28,1
6,2014,7,0.092,268.68,63.39,27.47,45.43,1
7,2014,8,0.111,272.92,69.19,19.73,44.46,1
8,2014,9,0.134,277.21,76.45,13.00,44.73,1
9,2014,10,0.161,281.08,82.35,8.25,45.30,1


> Реалізувати процедуру, яка змінить індекси областей, які використані на порталі NOAA на наступні: колонка `newID` фрейму `provinces`.

In [14]:
def renumber_provinces(provinceID, provinces): # https://stackoverflow.com/a/36685531
    return provinces.loc[provinces['provinceID'] == provinceID, 'newID'].iloc[0]

Продемонструємо використання, для області 5 новий індекс має дорівнювати 3:

In [15]:
print(renumber_provinces(5, provinces))

3


Тепер використаємо процедуру для всіх областей в `data_frame`:

In [16]:
# To understand apply() arguments, see https://stackoverflow.com/a/43460506
data_frame['Province'] = data_frame['Province'].apply(renumber_provinces, provinces=provinces)
data_frame

,Year,Week,SMN,SMT,VCI,TCI,VHI,Province
0,2014,1,0.070,264.20,59.34,26.13,42.74,22
1,2014,2,0.067,262.61,58.73,30.85,44.79,22
2,2014,3,0.066,261.66,57.73,35.26,46.49,22
3,2014,4,0.068,261.93,58.86,35.85,47.36,22
4,2014,5,0.073,263.23,60.34,34.37,47.35,22
5,2014,6,0.081,265.70,61.13,31.44,46.28,22
6,2014,7,0.092,268.68,63.39,27.47,45.43,22
7,2014,8,0.111,272.92,69.19,19.73,44.46,22
8,2014,9,0.134,277.21,76.45,13.00,44.73,22
9,2014,10,0.161,281.08,82.35,8.25,45.30,22


> Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):

> 1) Ряд VHI для області за рік, пошук екстремумів (min та max);

In [17]:
def province_year_stats(data_frame, province, year):
    stats = data_frame.loc[(data_frame['Province'] == province) & (data_frame['Year'] == year), ['Week', 'VHI']]
    return (stats['VHI'].min(), stats['VHI'].max())

province_year_stats(data_frame, 5, 2017)

(35.969999999999999, 59.840000000000003)

> 2) Ряд VHI за всі роки для області, виявити роки з екстремальними посухами, які торкнулися більше вказаного відсотка області;
* VHI &gt; 60 – сприятливі умови;
* VHI &lt; 40 – стресові умови;
* VHI &lt; 35 – посуха, інтенсивність якої від помірної до надзвичайної.
* VHI &lt; 15 – посуха, інтенсивність якої від середньої до надзвичайної;

In [21]:
def province_extreme_years(data_frame, province, extreme):
    years = data_frame.loc[(data_frame['Province'] == province) & (data_frame['VHI'] < extreme), ['Year']]
    return years.drop_duplicates()

province_extreme_years(data_frame, 1, 15.0)

,Year
85,2015


> 3) Аналогічно для помірних посух

In [22]:
province_extreme_years(data_frame, 1, 35.0)

,Year
82,2015
138,2016
166,2017
